In [2]:
import numpy as np
import struct

def read_nf(filename):
    # Open binary file
    # The file structure is defined in READNF
    with open(filename, 'rb') as f:
        # Read CSTAMP (26-char) and VERSNUM (int)
        # CSTAMP: 26 characters, then an integer
        # Fortran CHARACTER*26 likely stored as 26 bytes
        cstamp = f.read(26).decode('ascii', errors='ignore').strip()
        versnum = struct.unpack('>i', f.read(4))[0]  # adjust endianness if needed

        # if versnum not in [730, 731, 732, 733]:
        #     raise ValueError(f"Unsupported VERSNUM={versnum}")

        # Next, read header info per read in readnf.f90:
        # NRWORD_NF, NRFLDB, NXYZ, NAT0, NAT3, NCOMP, NX, NY, NZ
        # X0(3), AEFF, NAMBIENT, WAVE, AK_TF(3), CXE0_TF(3), CXB0_TF(3)
        # All are doubles except the ints. The Fortran code uses WP and checks NRWORD=8 for double precision.

        # Integers: NRWORD_NF, NRFLDB, NXYZ, NAT0, NAT3, NCOMP, NX, NY, NZ -> 9 ints
        data_ints = np.fromfile(f, dtype='>i4', count=9) # big-endian 4-byte int
        NRWORD_NF, NRFLDB, NXYZ, NAT0, NAT3, NCOMP, NX, NY, NZ = data_ints

        # Next doubles: X0(3), AEFF, NAMBIENT, WAVE, AK_TF(3), CXE0_TF(3 complex?), CXB0_TF(3 complex?)
        # CXE0_TF and CXB0_TF are complex. Fortran complex is stored as two doubles (real, imag).
        # The code reads them in one READ statement, so order is:
        # X0(3), AEFF, NAMBIENT, WAVE, AK_TF(3), CXE0_TF(3), CXB0_TF(3)
        # That's 3 + 1 + 1 + 1 + 3 = 9 doubles so far, plus 3 complex + 3 complex = 3*2+3*2=12 doubles more.
        # total doubles: 9 + 12 = 21 doubles
        data_dbl = np.fromfile(f, dtype='>f8', count=21)
        X0 = data_dbl[0:3]
        AEFF = data_dbl[3]
        NAMBIENT = data_dbl[4]
        WAVE = data_dbl[5]
        AK_TF = data_dbl[6:9]
        # Next 3 complex for CXE0_TF -> 6 doubles
        CXE0_TF = data_dbl[9:15].view(np.complex128)
        CXB0_TF = data_dbl[15:21].view(np.complex128)

        if NRWORD_NF != 8:
            # Expecting double precision = 8 bytes/word
            raise ValueError("Mismatch in word length")

        # Now read arrays:
        # CXEPS(NCOMP) complex
        # Each complex element: 2 doubles
        CXEPS = np.fromfile(f, dtype='>f8', count=2*NCOMP).view(np.complex128)

        # ICOMP(1:NX,1:NY,1:NZ,1:3) = 3*NXYZ integers
        ICOMP = np.fromfile(f, dtype='>i2', count=3*NXYZ)  # Fortran says INTEGER*2
        ICOMP = ICOMP.reshape((NX, NY, NZ, 3), order='F')

        # CXPOL, CXESCA, CXADIA each Nx,Ny,Nz,3 complex
        # Each complex value: 2 doubles
        def read_complex_array():
            arr = np.fromfile(f, dtype='>f8', count=2*NXYZ*3).view(np.complex128)
            return arr.reshape((NX, NY, NZ, 3), order='F')

        CXPOL = read_complex_array()
        CXESCA = read_complex_array()
        CXADIA = read_complex_array()

        CXBINC = CXBSCA = None
        if NRFLDB == 1:
            # CXBSCA(NX,NY,NZ,3)
            CXBSCA = read_complex_array()
            # For B fields, we will reconstruct CXBINC as done in code.
            # Actually, in READNF they do not store CXBINC but recompute it.
            # We'll do what READNF does: reconstruct CXBINC from CXB0_TF and phase
            # Instead of reading, we must compute once we have all info.

        # Close file handled by context manager

    # Recompute E_inc, B_inc, etc. as in readnf:
    # E_inc at each site = (3/(eps+2)) * E0 * phase_factor
    # same for B_inc from CXB0_TF
    # For simplicity, just replicate final arrays:
    # Build a phase factor grid:
    i_arr = np.arange(1, NX+1)
    j_arr = np.arange(1, NY+1)
    k_arr = np.arange(1, NZ+1)
    # Use broadcasting to build coordinates
    JX, JY, JZ = np.meshgrid(i_arr, j_arr, k_arr, indexing='ij')
    PHI0 = AK_TF[0]*X0[0] + AK_TF[1]*X0[1] + AK_TF[2]*X0[2]
    CXPHAS = np.exp(1j*( (JX*AK_TF[0]) + (JY*AK_TF[1]) + (JZ*AK_TF[2]) + PHI0 ))

    # Compute CXEINC:
    # need CXFAC=3/(CXEPS(IC)+2) at each point
    # ICOMP>0 means inside target
    # We'll just loop vectorized:
    # Get composition indices for each direction - just use the first direction:
    IC = ICOMP[...,0]
    # If IC>0, CXFAC=3/(CXEPS(IC)+2), else =1
    # For indexing CXEPS: IC start from 1? If yes, we must shift by -1 for zero-based indexing.
    # Note: If IC=0, no target. If >0, target composition. Assume 1-based indexing for compositions.
    CXFAC = np.ones((NX,NY,NZ,3), dtype=np.complex128)
    mask = (IC>0)
    CXFAC[mask,...] = 3.0/(CXEPS[IC[mask]-1]+2.0)
    # E_inc:
    CXEINC = CXFAC * CXE0_TF[np.newaxis,np.newaxis,np.newaxis,:] * CXPHAS[...,np.newaxis]

    # If NRFLDB=1, reconstruct B_inc similarly:
    if NRFLDB == 1:
        CXBINC = CXB0_TF[np.newaxis,np.newaxis,np.newaxis,:] * CXPHAS[...,np.newaxis]

    # Compute derived parameters
    DPHYS = AEFF*(4.*np.pi/(3.*NAT0))**(1./3.)
    XMIN = (X0[0]+1.-0.5001)*DPHYS
    XMAX = (X0[0]+NX+0.5001)*DPHYS
    YMIN = (X0[1]+1.-0.5001)*DPHYS
    YMAX = (X0[1]+NY+0.5001)*DPHYS
    ZMIN = (X0[2]+1.-0.5001)*DPHYS
    ZMAX = (X0[2]+NZ+0.5001)*DPHYS

    return {
        'cstamp': cstamp,
        'versnum': versnum,
        'NRFLDB': NRFLDB,
        'AEFF': AEFF,
        'DPHYS': DPHYS,
        'NX': NX, 'NY': NY, 'NZ': NZ,
        'NAT0': NAT0,
        'X0': X0,
        'XMIN': XMIN, 'XMAX': XMAX, 'YMIN': YMIN, 'YMAX': YMAX, 'ZMIN': ZMIN, 'ZMAX': ZMAX,
        'NAMBIENT': NAMBIENT,
        'WAVE': WAVE,
        'AK_TF': AK_TF,
        'CXE0_TF': CXE0_TF,
        'CXB0_TF': CXB0_TF,
        'NCOMP': NCOMP,
        'CXEPS': CXEPS,
        'ICOMP': ICOMP,
        'CXPOL': CXPOL,
        'CXESCA': CXESCA,
        'CXADIA': CXADIA,
        'CXEINC': CXEINC,     # reconstructed incident E
        'CXBINC': CXBINC,     # reconstructed incident B if NRFLDB=1
        'CXBSCA': (CXBSCA if NRFLDB==1 else None)
    }

# Example usage:
data = read_nf("./0-Final-3-45-Nearfield\w006r000k000.EB2")


ValueError: Mismatch in word length

In [ ]:
import numpy as np

def read_par_file(parfile):
    # Reads ddpostprocess.par format:
    # 1: cflename
    # 2: cflvtr (not used here, but we'll still read it)
    # 3: ivtr
    # 4: iline
    # subsequent lines: XA,YA,ZA,XB,YB,ZB,NAB per line until EOF
    with open(parfile, 'r') as f:
        cflename = f.readline().strip()
        # Even though we don't do VTR, we read the next line to maintain compatibility
        cflvtr = f.readline().strip()
        ivtr = int(f.readline().strip())
        iline = int(f.readline().strip())
        lines = []
        for line in f:
            vals = line.strip().split()
            if len(vals) == 7:
                XA, YA, ZA, XB, YB, ZB, NAB = vals
                XA, YA, ZA = float(XA), float(YA), float(ZA)
                XB, YB, ZB = float(XB), float(YB), float(ZB)
                NAB = int(NAB)
                lines.append((XA,YA,ZA,XB,YB,ZB,NAB))
    return cflename, ivtr, iline, lines

def trilinear_interpolation(XTF, data, X0, DPHYS):
    # data shape: (NX, NY, NZ, 3) for fields
    # XTF: (x,y,z) in physical units
    # Convert to fractional indices
    idx = XTF[0]/DPHYS - X0[0]
    jdy = XTF[1]/DPHYS - X0[1]
    kdz = XTF[2]/DPHYS - X0[2]

    NX, NY, NZ, _ = data.shape
    IX1 = int(np.floor(idx))
    IY1 = int(np.floor(jdy))
    IZ1 = int(np.floor(kdz))

    # clamp indices
    IX1 = max(0, min(IX1, NX-2))
    IY1 = max(0, min(IY1, NY-2))
    IZ1 = max(0, min(IZ1, NZ-2))

    WX = idx - IX1
    WY = jdy - IY1
    WZ = kdz - IZ1

    W1 = (1 - WX)*(1 - WY)*(1 - WZ)
    W2 = WX*(1 - WY)*(1 - WZ)
    W3 = (1 - WX)*WY*(1 - WZ)
    W4 = WX*WY*(1 - WZ)
    W5 = (1 - WX)*(1 - WY)*WZ
    W6 = WX*(1 - WY)*WZ
    W7 = (1 - WX)*WY*WZ
    W8 = WX*WY*WZ

    val = (W1*data[IX1, IY1, IZ1,:] +
           W2*data[IX1+1, IY1, IZ1,:] +
           W3*data[IX1, IY1+1, IZ1,:] +
           W4*data[IX1+1, IY1+1, IZ1,:] +
           W5*data[IX1, IY1, IZ1+1,:] +
           W6*data[IX1+1, IY1, IZ1+1,:] +
           W7*data[IX1, IY1+1, IZ1+1,:] +
           W8*data[IX1+1, IY1+1, IZ1+1,:])
    return val

def compute_snorm(CXE0_TF, CXB0_TF):
    # Compute 8*pi*|<S_inc>|
    ex, ey, ez = CXE0_TF
    bx, by, bz = CXB0_TF
    Sx = np.real(ey*np.conjugate(bz) - ez*np.conjugate(by))
    Sy = np.real(ez*np.conjugate(bx) - ex*np.conjugate(bz))
    Sz = np.real(ex*np.conjugate(by) - ey*np.conjugate(bx))
    SNORM = np.sqrt(Sx**2 + Sy**2 + Sz**2)
    return SNORM, np.array([Sx,Sy,Sz])

def compute_poynting(CXEINC, CXESCA, CXBINC, CXBSCA, SNORM):
    # Compute normalized Poynting vector S
    E_tot = CXEINC + CXESCA
    B_tot = CXBINC + CXBSCA
    B_conj = np.conjugate(B_tot)
    S = np.zeros_like(E_tot, dtype=float)
    S[...,0] = np.real(E_tot[...,1]*B_conj[...,2] - E_tot[...,2]*B_conj[...,1])/SNORM
    S[...,1] = np.real(E_tot[...,2]*B_conj[...,0] - E_tot[...,0]*B_conj[...,2])/SNORM
    S[...,2] = np.real(E_tot[...,0]*B_conj[...,1] - E_tot[...,1]*B_conj[...,0])/SNORM
    return S

def write_line_data(fnameE, fnameP, lines, data, NRFLDB, X0, DPHYS, SNORM, S_INC):
    # data dict: from read_nf
    # Interpolate along lines
    with open(fnameE, 'w') as fe, open(fnameP,'w') as fp:
        # Headers:
        if NRFLDB==0:
            fe.write("   x_TF       y_TF       z_TF    Re(E_x)   Im(E_x)  Re(E_y)  Im(E_y)  Re(E_z)  Im(E_z)\n")
        else:
            fe.write("                                    E field                                  B field                      \n")
            fe.write("   x_TF       y_TF       z_TF   Re(E_x) Im(E_x) Re(E_y) Im(E_y) Re(E_z) Im(E_z)  Re(B_x) Im(B_x) Re(B_y) Im(B_y) Re(B_z) Im(B_z)  Sx Sy Sz\n")
        fp.write("   x_TF       y_TF       z_TF    Re(P_x)  Im(P_x)  Re(P_y) Im(P_y) Re(P_z) Im(P_z)\n")

        CXEINC = data['CXEINC']
        CXESCA = data['CXESCA']
        CXPOL  = data['CXPOL']
        NRFLDB = data['NRFLDB']
        if NRFLDB == 1:
            CXBINC = data['CXBINC']
            CXBSCA = data['CXBSCA']
            # precompute Poynting vector on full grid
            S_grid = compute_poynting(CXEINC, CXESCA, CXBINC, CXBSCA, SNORM)
        else:
            S_grid = None

        for (XA,YA,ZA,XB,YB,ZB,NAB) in lines:
            # Check volume boundaries:
            if not (data['XMIN'] <= XA <= data['XMAX'] and data['XMIN'] <= XB <= data['XMAX'] and
                    data['YMIN'] <= YA <= data['YMAX'] and data['YMIN'] <= YB <= data['YMAX'] and
                    data['ZMIN'] <= ZA <= data['ZMAX'] and data['ZMIN'] <= ZB <= data['ZMAX']):
                print("Warning: line extends beyond computational volume, skipping line.")
                continue
            for JA in range(NAB+1):
                ZETA = JA/float(NAB)
                Xpos = XA+(XB - XA)*ZETA
                Ypos = YA+(YB - YA)*ZETA
                Zpos = ZA+(ZB - ZA)*ZETA
                pos = (Xpos, Ypos, Zpos)

                Einc_val = trilinear_interpolation(pos, CXEINC, data['X0'], data['DPHYS'])
                Esca_val = trilinear_interpolation(pos, CXESCA, data['X0'], data['DPHYS'])
                P_val    = trilinear_interpolation(pos, CXPOL,  data['X0'], data['DPHYS'])

                E_tot = Einc_val + Esca_val

                if NRFLDB==0:
                    Exr, Exi = np.real(E_tot[0]), np.imag(E_tot[0])
                    Eyr, Eyi = np.real(E_tot[1]), np.imag(E_tot[1])
                    Ezr, Ezi = np.real(E_tot[2]), np.imag(E_tot[2])
                    fe.write(f"{Xpos:10.3E} {Ypos:10.3E} {Zpos:10.3E} {Exr:10.5f} {Exi:10.5f} {Eyr:10.5f} {Eyi:10.5f} {Ezr:10.5f} {Ezi:10.5f}\n")
                else:
                    Binc_val = trilinear_interpolation(pos, data['CXBINC'], data['X0'], data['DPHYS'])
                    Bsca_val = trilinear_interpolation(pos, data['CXBSCA'], data['X0'], data['DPHYS'])
                    B_tot = Binc_val + Bsca_val
                    S_val = trilinear_interpolation(pos, S_grid, data['X0'], data['DPHYS'])

                    Exr, Exi = np.real(E_tot[0]), np.imag(E_tot[0])
                    Eyr, Eyi = np.real(E_tot[1]), np.imag(E_tot[1])
                    Ezr, Ezi = np.real(E_tot[2]), np.imag(E_tot[2])

                    Bxr, Bxi = np.real(B_tot[0]), np.imag(B_tot[0])
                    Byr, Byi = np.real(B_tot[1]), np.imag(B_tot[1])
                    Bzr, Bzi = np.real(B_tot[2]), np.imag(B_tot[2])

                    fe.write(f"{Xpos:10.3E} {Ypos:10.3E} {Zpos:10.3E} {Exr:10.5f} {Exi:10.5f} {Eyr:10.5f} {Eyi:10.5f} {Ezr:10.5f} {Ezi:10.5f} "
                             f"{Bxr:10.5f} {Bxi:10.5f} {Byr:10.5f} {Byi:10.5f} {Bzr:10.5f} {Bzi:10.5f} {S_val[0]:10.5f} {S_val[1]:10.5f} {S_val[2]:10.5f}\n")

                Pxr, Pxi = np.real(P_val[0]), np.imag(P_val[0])
                Pyr, Pyi = np.real(P_val[1]), np.imag(P_val[1])
                Pzr, Pzi = np.real(P_val[2]), np.imag(P_val[2])
                fp.write(f"{Xpos:10.3E} {Ypos:10.3E} {Zpos:10.3E} {Pxr:10.5f} {Pxi:10.5f} {Pyr:10.5f} {Pyi:10.5f} {Pzr:10.5f} {Pzi:10.5f}\n")

def write_3d_data(data):
    # Write out 3D data fields: XYZSCA.out, XYZINC.out, PART.out, POL.out, POLBI.out, EPS.out, OPPAR.out
    NX, NY, NZ = data['NX'], data['NY'], data['NZ']

    def write_complex_field(filename, field):
        with open(filename,'w') as f:
            for JX in range(NX):
                for JY in range(NY):
                    for JZ in range(NZ):
                        val = field[JX,JY,JZ,:]
                        f.write(f"{np.real(val[0]):E} {np.imag(val[0]):E} "
                                f"{np.real(val[1]):E} {np.imag(val[1]):E} "
                                f"{np.real(val[2]):E} {np.imag(val[2]):E}\n")

    # Scattered field
    write_complex_field('XYZSCA.out', data['CXESCA'])
    # Incoming field
    write_complex_field('XYZINC.out', data['CXEINC'])

    # Particle location:
    ICOMP_sum = np.sum(data['ICOMP'], axis=3)
    with open('PART.out','w') as f:
        for JX in range(NX):
            for JY in range(NY):
                for JZ in range(NZ):
                    val = ICOMP_sum[JX,JY,JZ]
                    f.write(f"{val:d}\n")

    # Polarization
    write_complex_field('POL.out', data['CXPOL'])

    # Polarizability
    write_complex_field('POLBI.out', data['CXADIA'])

    # Dielectric function
    with open('EPS.out','w') as f:
        val = data['CXEPS'][0]
        f.write(f"{np.real(val):E} {np.imag(val):E}\n")

    # OPPAR
    with open('OPPAR.out','w') as f:
        f.write(f"{NX}\n{NY}\n{NZ}\n")

def main():
    parfile = 'ddpostprocess.par'
    cflename, ivtr, iline, lines = read_par_file(parfile)

    # Read nearfield data
    from read_nf import read_nf  # Ensure read_nf.py is in same directory
    data = read_nf(cflename)

    # Compute SNORM
    SNORM, S_INC = compute_snorm(data['CXE0_TF'], data['CXB0_TF'])

    # If ILINE>0, write line data:
    if iline > 0:
        write_line_data('ddpostprocess.out', 'ddpostprocess_P.out', lines, data, data['NRFLDB'],
                        data['X0'], data['DPHYS'], SNORM, S_INC)

    # Write other 3D data files
    write_3d_data(data)

    print(">DDPOSTPROCESS: finished processing (no VTR/VTK output).")

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import struct

def readnf(filename, idvout=1):
    """
    Python equivalent of READNF subroutine from readnf.f90
    Returns:
        CSTAMP (str),
        VERSNUM (int),
        NRFLDB (int),
        AEFF (float),
        DPHYS (float),
        NX, NY, NZ (int),
        NAT0 (int),
        X0 (array of shape (3,)),
        XMIN,XMAX,YMIN,YMAX,ZMIN,ZMAX (float),
        NAMBIENT (float),
        WAVE (float),
        AK_TF (array (3,)),
        CXE0_TF (complex(3,)),
        CXB0_TF (complex(3,)),
        NCOMP (int),
        CXEPS (complex(NCOMP)),
        ICOMP (int array (NX,NY,NZ,3)),
        CXPOL (complex array (NX,NY,NZ,3)),
        CXESCA (complex array (NX,NY,NZ,3)),
        CXEINC (complex array (NX,NY,NZ,3)),  # computed after reading
        CXADIA (complex array (NX,NY,NZ,3)),
        CXBINC (complex array (NX,NY,NZ,3)) [if NRFLDB=1],
        CXBSCA (complex array (NX,NY,NZ,3)) [if NRFLDB=1].
    """

    # Open binary file
    with open(filename, 'rb') as f:
        # First read CSTAMP (26 chars) and VERSNUM (int)
        # Fortran: READ(IOBIN)CSTAMP,VERSNUM
        # CSTAMP is a 26-char string. In Fortran, CHARACTER*26 likely stored as 26 bytes.
        cstamp_bytes = f.read(26)
        CSTAMP = cstamp_bytes.decode('ascii', errors='replace').strip()
        # Next read VERSNUM (an integer)
        # Fortran default integer is likely 4 bytes
        VERSNUM = struct.unpack('i', f.read(4))[0]

        # Check version compatibility
        if VERSNUM not in [730, 731, 732, 733]:
            raise ValueError(f"Unsupported VERSNUM={VERSNUM}")

        # Next reads (based on the Fortran code):
        # NRWORD_NF (int), NRFLDB (int), NXYZ (int), NAT0 (int), NAT3 (int),
        # NCOMP (int), NX (int), NY (int), NZ (int),
        # X0(3) (3 doubles), AEFF (double), NAMBIENT (double), WAVE (double),
        # AK_TF(3) (3 doubles), CXE0_TF(3) (3 complex), CXB0_TF(3) (3 complex)
        # Real and complex likely double precision: 8 bytes each real part.
        # Complex stored as (real, imag) pairs.

        # Read integers:
        ints = np.fromfile(f, dtype=np.int32, count=9)
        NRWORD_NF, NRFLDB, NXYZ, NAT0, NAT3, NCOMP, NX, NY, NZ = ints

        # Determine float precision from NRWORD_NF
        # If NRWORD_NF=8: double precision
        # If NRWORD_NF=4: single precision
        # Based on code, likely double precision (8-byte) floats
        if NRWORD_NF == 8:
            float_dtype = np.float64
            complex_dtype = np.complex128
        elif NRWORD_NF == 4:
            float_dtype = np.float32
            complex_dtype = np.complex64
        else:
            raise ValueError("Unknown word length")

        # Read X0(3)
        X0 = np.fromfile(f, dtype=float_dtype, count=3)
        # AEFF, NAMBIENT, WAVE each double
        AEFF = np.fromfile(f, dtype=float_dtype, count=1)[0]
        NAMBIENT = np.fromfile(f, dtype=float_dtype, count=1)[0]
        WAVE = np.fromfile(f, dtype=float_dtype, count=1)[0]

        # AK_TF(3)
        AK_TF = np.fromfile(f, dtype=float_dtype, count=3)

        # CXE0_TF(3) complex and CXB0_TF(3) complex
        # Each complex number: 2 floats (real, imag)
        # We'll read them as real/imag pairs and reconstruct:
        def read_complex_array(count):
            arr = np.fromfile(f, dtype=float_dtype, count=2*count)
            return arr[::2] + 1j * arr[1::2]

        CXE0_TF = read_complex_array(3)
        CXB0_TF = read_complex_array(3)

        # Check NRWORD compatibility
        if NRWORD_NF != NRWORD_NF:  # redundant check, but keep logic
            raise ValueError("Mismatch in word length")

        # Now read CXEPS(NCOMP)
        CXEPS = read_complex_array(NCOMP)

        # ICOMP(NX,NY,NZ,3) integers
        # total count = NX*NY*NZ*3 integers
        ICOMP = np.fromfile(f, dtype=np.int32, count=NX*NY*NZ*3)
        ICOMP = ICOMP.reshape((NX,NY,NZ,3), order='F') # Fortran order guess

        # CXPOL(NX,NY,NZ,3) complex
        # count of complex elements = NX*NY*NZ*3
        CXPOL_arr = read_complex_array(NX*NY*NZ*3)
        CXPOL = CXPOL_arr.reshape((NX,NY,NZ,3), order='F')

        # CXESCA(NX,NY,NZ,3) complex
        CXESCA_arr = read_complex_array(NX*NY*NZ*3)
        CXESCA = CXESCA_arr.reshape((NX,NY,NZ,3), order='F')

        # CXADIA(NX,NY,NZ,3) complex
        CXADIA_arr = read_complex_array(NX*NY*NZ*3)
        CXADIA = CXADIA_arr.reshape((NX,NY,NZ,3), order='F')

        CXBINC = None
        CXBSCA = None
        if NRFLDB == 1:
            # CXBSCA(NX,NY,NZ,3) complex
            CXBSCA_arr = read_complex_array(NX*NY*NZ*3)
            CXBSCA = CXBSCA_arr.reshape((NX,NY,NZ,3), order='F')
        # file closed by context manager

    # Compute derived quantities
    PI = 4.0*np.arctan(1.0)
    DPHYS = AEFF * (4.0*PI/(3.0*NAT0))**(1.0/3.0)
    XMIN = (X0[0] + 1. - 0.5001)*DPHYS
    XMAX = (X0[0] + NX + 0.5001)*DPHYS
    YMIN = (X0[1] + 1. - 0.5001)*DPHYS
    YMAX = (X0[1] + NY + 0.5001)*DPHYS
    ZMIN = (X0[2] + 1. - 0.5001)*DPHYS
    ZMAX = (X0[2] + NZ + 0.5001)*DPHYS

    # Recompute CXEINC and CXBINC
    # CXEINC and CXBINC depend on phase factor = exp(i*(JX*AK_TF(1)+JY*AK_TF(2)+JZ*AK_TF(3) + PHI0))
    # PHI0 = AK_TF(1)*X0(1)+AK_TF(2)*X0(2)+AK_TF(3)*X0(3)
    PHI0 = AK_TF[0]*X0[0] + AK_TF[1]*X0[1] + AK_TF[2]*X0[2]

    CXEINC = np.zeros((NX,NY,NZ,3), dtype=complex_dtype)
    if NRFLDB == 1:
        CXBINC = np.zeros((NX,NY,NZ,3), dtype=complex_dtype)

    for jz in range(NZ):
        PHIZ = PHI0 + (jz+1)*AK_TF[2] # Note: Fortran indexing starts at 1, adapt as needed
        for jy in range(NY):
            PHIYZ = PHIZ + (jy+1)*AK_TF[1]
            for jx in range(NX):
                phase = np.exp(1j*((jx+1)*AK_TF[0]+PHIYZ))
                for k in range(3):
                    ic = ICOMP[jx,jy,jz,k]
                    cxfac = 1.0
                    if ic > 0:
                        cxfac = 3.0/(CXEPS[ic-1]+2.0)  # ic-1 since Python is 0-based
                    CXEINC[jx,jy,jz,k] = cxfac*CXE0_TF[k]*phase
                if NRFLDB == 1:
                    for k in range(3):
                        CXBINC[jx,jy,jz,k] = CXB0_TF[k]*phase

    # Optional checks and logging as in the Fortran code
    # For simplicity, just print some info:
    print(f">READNF data from {CSTAMP}")
    print(f">READNF lambda={WAVE} physical units")
    print(f">READNF  aeff ={AEFF} physical units")
    print(f">READNF  NAT0 ={NAT0} target dipoles")
    print(f">READNF    d  ={DPHYS} physical units")
    print(f">READNF target xmin,xmax={XMIN},{XMAX} physical units")
    print(f">READNF target ymin,ymax={YMIN},{YMAX} physical units")
    print(f">READNF target zmin,zmax={ZMIN},{ZMAX} physical units")

    # Compute solution error check as Fortran does (optional)
    EINC2 = np.sum(CXE0_TF * np.conjugate(CXE0_TF))
    SUMERR2 = 0.0+0j
    for jz in range(NZ):
        for jy in range(NY):
            for jx in range(NX):
                # Check if occupied:
                if any(ICOMP[jx,jy,jz,:]>0):
                    for k in range(3):
                        ic = ICOMP[jx,jy,jz,k]
                        if ic>0:
                            CXERR = CXPOL[jx,jy,jz,k]*CXADIA[jx,jy,jz,k] - \
                                    (CXEINC[jx,jy,jz,k] + CXESCA[jx,jy,jz,k])* (CXEPS[ic-1]+2.0)/3.0
                            SUMERR2 += CXERR*np.conjugate(CXERR)
    SUMERR2 = SUMERR2/(NAT0*EINC2)
    print(f">READNF {SUMERR2} = normalized error |P/alpha-E|^2/|E_inc|^2")

    # Return all values
    return (CSTAMP, VERSNUM, NRFLDB, AEFF, DPHYS, NX, NY, NZ, NAT0, X0, XMIN, XMAX,
            YMIN, YMAX, ZMIN, ZMAX, NAMBIENT, WAVE, AK_TF, CXE0_TF, CXB0_TF, NCOMP,
            CXEPS, ICOMP, CXPOL, CXESCA, CXEINC, CXADIA, CXBINC, CXBSCA)


# Example usage:
# CSTAMP, VERSNUM, NRFLDB, AEFF, DPHYS, NX, NY, NZ, NAT0, X0, \
# XMIN, XMAX, YMIN, YMAX, ZMIN, ZMAX, NAMBIENT, WAVE, AK_TF, CXE0_TF, CXB0_TF, \
# NCOMP, CXEPS, ICOMP, CXPOL, CXESCA, CXEINC, CXADIA, CXBINC, CXBSCA = readnf('ddpostprocess.bin')
